In [ ]:
#Packages
import openpyxl
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
import subprocess
import numpy as np
#Helper functions
def plot_scatter_mul(dataframe, guide, title, xlabel, ylabel, legend_label):
    i = 0
    if guide == 'x':
        for index, row in df.iterrows():
            if index % 2 != 0: 
                i+= 1
                plt.scatter(list(element + 1 for element in list(range(df.shape[1]))), df.iloc[index], label=legend_label + f'{i}')
    elif guide == 'y':
        for index, row in df.iterrows():
            if index % 2 == 0: 
                i+= 1
                plt.scatter(list(element + 1 for element in list(range(df.shape[1]))), df.iloc[index], label=legend_label + f'{i}')
    else: 
        raise Exception('Select the fiber x or y')
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.show()

def RyR(dataframe, guide):
    i = 0
    dfx = []
    dfy = []
    std = dataframe.iloc[:, :-2].std(axis=1) #Standard deviation for each row, excluding limits
    if guide == 'x':
        for index, _ in enumerate(range(dataframe.shape[0])):
            if index % 2 != 0: 
                i+= 1
                RyR = (6*std.iloc[index]/(dataframe.iloc[index,-1]-dataframe.iloc[index,-2]))*100
                Pass = "Correcto" if RyR <= 10 else ("Fallo ligero" if 10 <= RyR <= 25 else "Fallo considerable")
                print(f'RyR Guía fbx {i}: ' + str(RyR) +' %' + ' Estatus: ' + Pass)
                dfx.append(float(RyR))
        dfx = pd.Series(dfx)
        print("")
        return dfx
    elif guide == 'y':
        for index, _ in enumerate(range(dataframe.shape[0])):
            if index % 2 == 0: 
                i+= 1
                RyR = (6*std.iloc[index]/(dataframe.iloc[index,-1]-dataframe.iloc[index,-2]))*100
                Pass = "Correcto" if RyR <= 10 else ("Fallo ligero" if 10 <= RyR <= 25 else "Fallo considerable")
                print(f'RyR Guía fby {i}: ' + str(RyR) +' %' + ' Estatus: ' + Pass)
                dfy.append(RyR)
        dfy = pd.Series(dfy)
        print("")
        return dfy
    else: 
        raise Exception('Select the fiber x or y')

def z_score_filter(dataframe, threshold):
    rows = []
    MEAS = dataframe.iloc[:, :-2]
    limits = dataframe.iloc[:, -2:]    
    for row in range(MEAS.shape[0]):
        row = MEAS.iloc[row, :]
        z_scores = (row - row.mean()) / row.std()
        filtered_row = np.where(abs(z_scores) <= threshold, row, np.nan)
        rows.append(filtered_row)
    filtered_df = pd.DataFrame(rows)
    filtered_df = pd.concat([filtered_df, limits], axis=1)
    return filtered_df

Data Extraction and visualization

In [ ]:
#Imports the data from a prepocessed working sheet
data = pd.read_excel("./2_Results/Target.xlsx", header=None)
#Slice the measure from the data
df = data.iloc[3:, 1:-2] #Slices the dataframe, deleting the first empty rows and the columns for tags and df_plus_limits
df = df.reset_index(drop=True) #Resets the rows index to start in zero again
df.columns = range(df.shape[1]) #Resets the columns index to start in zero again
#Plots the measures for each scale
plot_scatter_mul(df, 'x', 'Scatter Plot, fiber X', 'test', 'MEAS', f'Guía fbx:')
plot_scatter_mul(df, 'y', 'Scatter Plot, fiber Y', 'test', 'MEAS', f'Guía fby:')

RyR calculation

In [ ]:
#Slices the dataframe to obtain only the measures and limits for each fiber
df_plus_limits = pd.concat([df, data.iloc[3:, -2:].reset_index(drop=True)], axis=1) #Reset the index to avoid index mismatching
df_plus_limits = df_plus_limits.reset_index(drop=True) #Resets the rows index to start in zero again
df_plus_limits.columns = range(df_plus_limits.shape[1]) #Resets the columns index to start in zero again
#Generates a RyR for each fiber in every scale
dfx = RyR(df_plus_limits, 'x')
dfy = RyR(df_plus_limits, 'y')

Filtering

In [ ]:
#Filter values by their row z-score
z_df = z_score_filter(df_plus_limits, 3)
#Split the dataframe for x fiber and y fiber
x_fiber = z_df[z_df.index % 2 == 0]
y_fiber = z_df[z_df.index % 2 != 0]
x_df = pd.DataFrame(z_df, 'x')



dfx = RyR(x_df, 'x')
dfy = RyR(y_df, 'y')


In [ ]:
def RyR(dataframe, guide):
    i = 0; dfx = []; dfy = []
    std = dataframe.iloc[:, :-2].std(axis=1) #Standard deviation for each row, excluding limits
    for index, _ in enumerate(range(dataframe.shape[0])):
        i+= 1
        RyR = (6*std.iloc[index]/(dataframe.iloc[index,-1]-dataframe.iloc[index,-2]))*100
        Pass = "Correcto" if RyR <= 10 else ("Fallo ligero" if 10 <= RyR <= 25 else "Fallo considerable")
        df_RyR.append(float(RyR))
    x_fiber = df_RyR[df_RyR.index % 2 == 0]
    y_fiber = df_RyR[df_RyR.index % 2 != 0]
    dfx = pd.Series(x_fiber)
    dfy = pd.Series(y_fiber)
    print(f'RyR Guía fbx {i}: ' + str(RyR) +' %' + ' Estatus: ' + Pass)
    print("")
    print(f'RyR Guía fby {i}: ' + str(RyR) +' %' + ' Estatus: ' + Pass)
    return dfx, dfy

dfx = RyR(x_df, 'x')
dfy = RyR(y_df, 'y')
